In [1]:
import torch
import numpy as np

In [4]:
import sys
sys.path.append('./base_model')

In [10]:
from base_model.dataset import wrap_dataset
from base_model.model import DisentangleVAE
from interface import PolyDisVAE

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
def load_sample(path):
    dataset = wrap_dataset([path], [0], shift_low=0, shift_high=0,
                           num_bar=2, contain_chord=True)
    mel, prs, pr_mat, x, c, dt_x = dataset[0]
    return x, c, pr_mat

In [7]:
def prepare_tensors(x, c, pr_mat, device):
    x = torch.tensor(x).long().unsqueeze(0).to(device)
    c = torch.tensor(c).float().unsqueeze(0).to(device)
    pr_mat = torch.tensor(pr_mat).float().unsqueeze(0).to(device)
    return x, c, pr_mat

In [23]:
def load_model(device):
    interface = PolyDisVAE.init_model(device=device)
    interface.load_model('./model_param/polydis-v1.pt')
    model = DisentangleVAE('disvae', device,
                           interface.chd_encoder,
                           interface.txt_encoder,
                           interface.pnotree_decoder,
                           interface.chd_decoder)
    model.eval()
    return model

In [24]:
load_model(device)

DisentangleVAE(
  (chd_encoder): ChordEncoder(
    (gru): GRU(36, 1024, batch_first=True, bidirectional=True)
    (linear_mu): Linear(in_features=2048, out_features=256, bias=True)
    (linear_var): Linear(in_features=2048, out_features=256, bias=True)
  )
  (rhy_encoder): TextureEncoder(
    (cnn): Sequential(
      (0): Conv2d(1, 10, kernel_size=(4, 12), stride=(4, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    )
    (fc1): Linear(in_features=290, out_features=1000, bias=True)
    (fc2): Linear(in_features=1000, out_features=256, bias=True)
    (gru): GRU(256, 1024, batch_first=True, bidirectional=True)
    (linear_mu): Linear(in_features=2048, out_features=256, bias=True)
    (linear_var): Linear(in_features=2048, out_features=256, bias=True)
  )
  (decoder): PianoTreeDecoder(
    (note_embedding): Linear(in_features=135, out_features=128, bias=True)
    (z2dec_hid_linear): Linear(in_features=512, out_feature

In [25]:
def calc_loss():
    data_path = './dataSet/POP09-PIANOROLL-4-bin-quantization/001.npz'
    model = load_model(device=device)
    x, c, pr_mat = load_sample(data_path)
    x, c, pr_mat = prepare_tensors(x, c, pr_mat, model.device)

    loss, *_ = model.loss(x, c, pr_mat)
    print('Loss:', loss.item())

In [26]:
calc_loss()

Loss: 0.4220174551010132


In [27]:
from compute_single_loss import main as compute_single 

In [28]:
main()

Loss: 0.4694575071334839
